# He Weight initialization, Forward & Backward Propagation

## Variance equations 

\begin{align}
    \operatorname{Var}(X) = \operatorname{E}\left[X^2 \right] - \operatorname{E}[X]^2 \label{eq1}\tag{1} \\
\end{align}

The Variance of the product of two independent random variables:

\begin{align}
\operatorname{Var}(X \cdot Y) = \operatorname{Var}(X) \cdot \operatorname{Var}(Y) + \operatorname{Var}(X) \cdot \operatorname{E}(X)^2 + \operatorname{E}(X)^2 \cdot \operatorname{Var}(Y) \label{eq2}\tag{2}
\end{align}

## Forward Propagation
\begin{align}
y_l = W_l x_l + b_l \label{eq3}\tag{3}\\
\end{align}

### Variance of the *l*-th layer output
In ideal case, mean is 0 and std is 1

$ \operatorname{Var}(y_l) = n_l \cdot \operatorname{Var}(w_l \cdot x_l)$

$ w_l $ has zero mean

According to ([2](#mjx-eqn-eq2)), the variance of the product of independent variables gives us:
\begin{align}
 \operatorname{Var}(y_l) = n_l \cdot \operatorname{Var}(w_l) \cdot \operatorname{E}(x_l)^2 \label{eq4}\tag{4}\\
\end{align}

For ReLU activation $ x_l = \operatorname{max}(0, y_{l-1}) $, $x_l$ does not have zero mean.

$ \boldsymbol{x_l} = \operatorname{f}(\boldsymbol{y}_{l-1}) $

When *f* is ReLU, $\operatorname{E}(x_l^2) = \frac{1}{2}n_l\operatorname{Var}(y_{l-1}) $

With *l* layers put together, we have:
\begin{align}
\operatorname{Var}(y_L) = \operatorname{Var}(y_1) \cdot \left(\prod_{l=2}^{L} \frac{1}{2} n_l \operatorname{Var}(w_l)\right) \label{eq5}\tag{5}
\end{align}

The product is the key to the initialization design. A proper initialization method should avoid reducing or magnifying the magnitudes of input signals exponentially. So we expect the above product to take a properscalar (e.g., 1). A sufficient condition is:
    \begin{align}
    \frac{1}{2}n_l\operatorname{Var}(w_l) = 1, 	\forall{l} \label{eq6}\tag{6}
    \end{align}

Thus,

\begin{align}
\operatorname{Var}(w_l) = \frac{2}{n_l} \\
\operatorname{std}(w_l) = \sqrt{\frac{2}{n_l}} \label{eq7}\tag{7}
\end{align}

For the first layer $(l = 1)$, we should have $n_l\operatorname{Var}(w_l)=1$ because there is no ReLU applied to the input signal. But thefactor $\frac{1}{2}$ does not matter if it justexists on one layer, so we adopt ([7](#mjx-eqn-eq7)) in the first layer for simplicity.

## Backward Propagation
\begin{align}
\Delta \bf{x_l} = \hat{\bf{W_l}}\bf{y_l} \\
\Delta \bf{x} = \frac{\partial \epsilon}{\partial \bf{x}} \\
\Delta \bf{y} = \frac{\partial \epsilon}{\partial \bf{y}} \\
\end{align}

$\Delta\bf{y}$ represents *k*-by-*k* pixels in *d* channels, and is reshaped into a $k^2d$-by-1 vector.

# PyTorch Implementation

In [14]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [15]:
#export
from exp.nb_01 import *

def get_data():
    path = datasets.download_data(MNIST_URL, ext='.gz')
    with gzip.open(path, 'rb') as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
    return map(tensor, (x_train,y_train,x_valid,y_valid))

def normalize(x, m, s): return (x-m)/s

In [16]:
x_train,y_train,x_valid,y_valid = get_data()

In [17]:
train_mean,train_std = x_train.mean(),x_train.std()
train_mean,train_std

(tensor(0.1304), tensor(0.3073))

## Data Normalization

In [19]:
x_train = normalize(x_train, train_mean, train_std)
# NB: Use training, not validation mean for validation set
x_valid = normalize(x_valid, train_mean, train_std)

In [20]:
train_mean,train_std = x_train.mean(),x_train.std()
train_mean,train_std

(tensor(0.0001), tensor(1.))

In [21]:
#export
def test_near_zero(a,tol=1e-3): assert a.abs()<tol, f"Near zero: {a}"

In [22]:
test_near_zero(x_train.mean())
test_near_zero(1-x_train.std())

In [23]:
n,m = x_train.shape
c = y_train.max()+1
n,m,c

(50000, 784, tensor(10))

## Basic architecture

In [24]:
# num hidden
nh = 50

In [25]:
# simplified kaiming init / he init
w1 = torch.randn(m,nh)/math.sqrt(m)
b1 = torch.zeros(nh)
w2 = torch.randn(nh,1)/math.sqrt(nh)
b2 = torch.zeros(1)

In [26]:
test_near_zero(w1.mean())
test_near_zero(w1.std()-1/math.sqrt(m))

In [27]:
# This should be ~ (0,1) (mean,std)...
x_valid.mean(),x_valid.std()

(tensor(-0.0057), tensor(0.9924))

In [28]:
def lin(x, w, b): return x@w + b

In [29]:
t = lin(x_valid, w1, b1)

In [31]:
t.mean(), t.std()

(tensor(-0.0150), tensor(0.9900))

In [35]:
def relu(x): return x.clamp_min(0.)

In [36]:
t = relu(lin(x_valid, w1, b1))

In [37]:
t.mean(), t.std()

(tensor(0.3820), tensor(0.5882))

From pytorch docs: `a: the negative slope of the rectifier used after this layer (0 for ReLU by default)`

$$\text{std} = \sqrt{\frac{2}{(1 + a^2) \times \text{fan_in}}}$$

This was introduced in the paper that described the Imagenet-winning approach from *He et al*: [Delving Deep into Rectifiers](https://arxiv.org/abs/1502.01852), which was also the first paper that claimed "super-human performance" on Imagenet (and, most importantly, it introduced resnets!)

In [38]:
# w1 kaiming init / e init for ReLU
w1 = torch.randn(m, nh)*math.sqrt(2/m)

In [39]:
w1.mean(), w1.std()

(tensor(-0.0004), tensor(0.0508))

In [40]:
t = relu(lin(x_valid, w1, b1))
t.mean(),t.std()

(tensor(0.4526), tensor(0.7556))

In [41]:
#export
from torch.nn import init

In [42]:
w1 = torch.zeros(m, nh)
init.kaiming_normal_(w1, mode='fan_out')
t = relu(lin(x_valid, w1, b1))

In [44]:
init.kaiming_normal_??

Signature: init.kaiming_normal_(tensor, a=0, mode='fan_in', nonlinearity='leaky_relu')
Source:   
def kaiming_normal_(tensor, a=0, mode='fan_in', nonlinearity='leaky_relu'):
    r"""Fills the input `Tensor` with values according to the method
    described in `Delving deep into rectifiers: Surpassing human-level
    performance on ImageNet classification` - He, K. et al. (2015), using a
    normal distribution. The resulting tensor will have values sampled from
    :math:`\mathcal{N}(0, \text{std}^2)` where

    .. math::
        \text{std} = \sqrt{\frac{2}{(1 + a^2) \times \text{fan\_in}}}

    Also known as He initialization.

    Args:
        tensor: an n-dimensional `torch.Tensor`
        a: the negative slope of the rectifier used after this layer (0 for ReLU
            by default)
        mode: either ``'fan_in'`` (default) or ``'fan_out'``. Choosing ``'fan_in'``
            preserves the magnitude of the variance of the weights in the
            forward pass. Choosing ``'fan_

In [45]:
w1.mean(), w1.std()

(tensor(0.0002), tensor(0.0507))

In [46]:
t.mean(), t.std()

(tensor(0.5699), tensor(0.8459))

In [48]:
w1.shape

torch.Size([784, 50])

In [49]:
import torch.nn

In [50]:
torch.nn.Linear(m, nh).weight.shape

torch.Size([50, 784])

This is why we set mode='fan_out' => the linear transformation performed by Linear is 'y = xA^T + b' as opposed to 'y = xA + b' defined above

In [51]:
torch.nn.Linear.forward??

Signature: torch.nn.Linear.forward(self, input)
Docstring:
Defines the computation performed at every call.

Should be overridden by all subclasses.

.. note::
    Although the recipe for forward pass needs to be defined within
    this function, one should call the :class:`Module` instance afterwards
    instead of this since the former takes care of running the
    registered hooks while the latter silently ignores them.
Source:   
    def forward(self, input):
        return F.linear(input, self.weight, self.bias)
File:      /opt/anaconda3/envs/fastai/lib/python3.6/site-packages/torch/nn/modules/linear.py
Type:      function


In [52]:
torch.nn.functional.linear??

Signature: torch.nn.functional.linear(input, weight, bias=None)
Source:   
def linear(input, weight, bias=None):
    # type: (Tensor, Tensor, Optional[Tensor]) -> Tensor
    r"""
    Applies a linear transformation to the incoming data: :math:`y = xA^T + b`.

    Shape:

        - Input: :math:`(N, *, in\_features)` where `*` means any number of
          additional dimensions
        - Weight: :math:`(out\_features, in\_features)`
        - Bias: :math:`(out\_features)`
        - Output: :math:`(N, *, out\_features)`
    """
    if input.dim() == 2 and bias is not None:
        # fused op is marginally faster
        ret = torch.addmm(bias, input, weight.t())
    else:
        output = input.matmul(weight.t())
        if bias is not None:
            output += bias
        ret = output
    return ret
File:      /opt/anaconda3/envs/fastai/lib/python3.6/site-packages/torch/nn/functional.py
Type:      function


In [53]:
torch.nn.Conv2d??

Init signature:
torch.nn.Conv2d(
    in_channels,
    out_channels,
    kernel_size,
    stride=1,
    padding=0,
    dilation=1,
    groups=1,
    bias=True,
    padding_mode='zeros',
)
Source:        
class Conv2d(_ConvNd):
    r"""Applies a 2D convolution over an input signal composed of several input
    planes.

    In the simplest case, the output value of the layer with input size
    :math:`(N, C_{\text{in}}, H, W)` and output :math:`(N, C_{\text{out}}, H_{\text{out}}, W_{\text{out}})`
    can be precisely described as:

    .. math::
        \text{out}(N_i, C_{\text{out}_j}) = \text{bias}(C_{\text{out}_j}) +
        \sum_{k = 0}^{C_{\text{in}} - 1} \text{weight}(C_{\text{out}_j}, k) \star \text{input}(N_i, k)


    where :math:`\star` is the valid 2D `cross-correlation`_ operator,
    :math:`N` is a batch size, :math:`C` denotes a number of channels,
    :math:`H` is a height of input planes in pixels, and :math:`W` is
    width in pixels.

    * :attr:`stride` controls the st

In [54]:
torch.nn.modules.conv._ConvNd.reset_parameters??

Signature: torch.nn.modules.conv._ConvNd.reset_parameters(self)
Docstring: <no docstring>
Source:   
    def reset_parameters(self):
        init.kaiming_uniform_(self.weight, a=math.sqrt(5))
        if self.bias is not None:
            fan_in, _ = init._calculate_fan_in_and_fan_out(self.weight)
            bound = 1 / math.sqrt(fan_in)
            init.uniform_(self.bias, -bound, bound)
File:      /opt/anaconda3/envs/fastai/lib/python3.6/site-packages/torch/nn/modules/conv.py
Type:      function


In [55]:
# Jeremy's ReLU
def relu(x): return x.clamp_min(0.) - 0.5

In [88]:
# kaiming init / he init for relu
w1 = torch.randn(m, nh)*math.sqrt(2/m)
t1 = relu(lin(x_valid, w1, b1))
t1.mean(), t1.std()

(tensor(0.0449), tensor(0.7865))

In [57]:
def model(xb):
    l1 = lin(xb, w1, b1)
    l2 = relu(l1)
    l3 = lin(l2, w2, b2)
    return l3

In [58]:
%timeit -n 10 _=model(x_valid)

6.55 ms ± 1.25 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [59]:
assert model(x_valid).shape==torch.Size([x_valid.shape[0],1])

### Loss function: MSE

In [60]:
model(x_valid).shape

torch.Size([10000, 1])

We need `squeeze()` to get rid of that trailing (,1), in order to use `mse`. (Of course, `mse` is not a suitable loss function for multi-class classification; we'll use a better loss function soon. We'll use `mse` for now to keep things simple.)

In [67]:
def mse(output, targ): return (output.squeeze(-1) - targ).pow(2).mean()

In [63]:
y_train, y_valid = y_train.float(), y_train.float()

In [64]:
preds = model(x_train)

In [65]:
preds.shape, y_train.shape

(torch.Size([50000, 1]), torch.Size([50000]))

In [68]:
mse(preds, y_train)

tensor(31.1430)

In [75]:
def mse_grad(inp, targ):
    #grad loss with respect to output of previous layer
    inp.g = 2. * (inp.squeeze() - targ).unsqueeze(-1)/inp.shape[0]

In [70]:
def relu_grad(inp, out):
    # grad relu with respect to input activations
    inp.g = (inp > 0).float() * out.g

In [79]:
def lin_grad(inp, out, w, b):
    # grad of matmul with respect to input
    inp.g = out.g @ w.t()
    w.g = (inp.unsqueeze(-1) * out.g.unsqueeze(1)).sum(0)
    b.g = out.g.sum(0)

In [73]:
def forward_and_backward(inp, targ):
    l1 = inp @ w1 + b1
    l2 = relu(l1)
    out = l2 @ w2 + b2
    
    #we actually don't need loss in backward!
    loss= mse(out, targ)
    
    #backward pass
    mse_grad(out,targ)
    lin_grad(l2, out, w2, b2)
    relu_grad(l1, l2)
    lin_grad(inp, l1, w1, b1)

In [89]:
forward_and_backward(x_train, y_train)

In [90]:
# Save for testing against later
w1g = w1.g.clone()
w2g = w2.g.clone()
b1g = b1.g.clone()
b2g = b2.g.clone()
ig  = x_train.g.clone()

We cheat a little bit and use PyTorch autograd to check our results.

In [91]:
xt2 = x_train.clone().requires_grad_(True)
w12 = w1.clone().requires_grad_(True)
w22 = w2.clone().requires_grad_(True)
b12 = b1.clone().requires_grad_(True)
b22 = b2.clone().requires_grad_(True)

In [92]:
def forward(inp, targ):
    # forward pass:
    l1 = inp @ w12 + b12
    l2 = relu(l1)
    out = l2 @ w22 + b22
    # we don't actually need the loss in backward!
    return mse(out, targ)

In [93]:
loss = forward(xt2, y_train)

In [94]:
loss.backward()

In [95]:
test_near(w22.grad, w2g)
test_near(b22.grad, b2g)
test_near(w12.grad, w1g)
test_near(b12.grad, b1g)
test_near(xt2.grad, ig )

## Refactor model

### Layers as classes

In [96]:
class Relu():
    def __call__(self, inp):
        self.inp = inp
        self.out = inp.clamp_min(0.) - 0.5
        return self.out
    def backward(self): self.inp.g = (self.inp > 0).float() * self.out.g

In [97]:
class Lin():
    def __init__(self, w, b): self.w, self.b = w, b
    
    def __call__(self, inp):
        self.inp = inp
        self.out = inp@self.w + self.b
        return self.out
    
    def backward(self):
        self.inp.g = self.out.g @ self.w.t()
        self.w.g = (self.inp.unsqueeze(-1) * self.out.g.unsqueeze(1)).sum(0)
        self.b.g = self.out.g.sum(0)

In [104]:
class Mse():
    def __call__(self, inp, targ):
        self.inp = inp
        self.targ = targ
        self.out = (inp.squeeze() - targ).pow(2).mean()
        return self.out
    
    def backward(self):
        self.inp.g = 2. * (self.inp.squeeze() - self.targ).unsqueeze(-1) / self.targ.shape[0]

In [106]:
class Model():
    def __init__(self, w1, b1, w2, b2):
        self.layers = [Lin(w1, b1), Relu(), Lin(w2, b2)]
        self.loss = Mse()
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()

In [107]:
w1.g,b1.g,w2.g,b2.g = [None]*4
model = Model(w1, b1, w2, b2)

In [108]:
%time loss = model(x_train, y_train)

CPU times: user 165 ms, sys: 4.36 ms, total: 170 ms
Wall time: 45 ms


In [109]:
%time model.backward()

CPU times: user 8.78 s, sys: 12.9 s, total: 21.7 s
Wall time: 21.8 s


In [110]:
test_near(w2g, w2.g)
test_near(b2g, b2.g)
test_near(w1g, w1.g)
test_near(b1g, b1.g)
test_near(ig, x_train.g)

### Module.forward()

In [111]:
class Module():
    def __call__(self, *args):
        self.args = args
        self.out = self.forward(*args)
        return self.out
    
    def forward(self): raise Exception('not implemented')
    def backward(self): self.bwd(self.out, *self.args)

In [112]:
class Relu(Module):
    def forward(self, inp): return inp.clamp_min(0.) - 0.5
    def bwd(self, out, inp): inp.g = (inp > 0).float() * out.g

In [114]:
class Lin(Module):
    def __init__(self, w, b): self.w, self.b = w, b
        
    def forward(self, inp): return inp @ self.w + self.b
    
    def bwd(self, out, inp):
        inp.g = out.g @ self.w.t()
        self.w.g = torch.einsum("bi,bj->ij", inp, out.g)
        self.b.g = out.g.sum(0)

In [115]:
class Mse(Module):
    def forward(self, inp, targ): return (inp.squeeze() - targ).pow(2).mean()
    def bwd(self, out, inp, targ): inp.g = 2*(inp.squeeze() - targ).unsqueeze(-1) / targ.shape[0]

In [119]:
class Model():
    def __init__(self):
        self.layers = [Lin(w1, b1), Relu(), Lin(w2, b2)]
        self.loss = Mse()
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()

In [120]:
w1.g,b1.g,w2.g,b2.g = [None]*4
model = Model()

In [121]:
%time loss = model(x_train, y_train)

CPU times: user 165 ms, sys: 4.83 ms, total: 170 ms
Wall time: 45.5 ms


In [122]:
%time model.backward()

CPU times: user 539 ms, sys: 89.9 ms, total: 629 ms
Wall time: 167 ms


In [123]:
test_near(w2g, w2.g)
test_near(b2g, b2.g)
test_near(w1g, w1.g)
test_near(b1g, b1.g)
test_near(ig, x_train.g)

### Without einsum

In [131]:
class Lin(Module):
    def __init__(self, w, b): self.w, self.b = w, b
        
    def forward(self,inp): return inp @ self.w + self.b
    
    def bwd(self, out, inp):
        inp.g = out.g @ self.w.t()
        self.w.g = inp.t() @ out.g
        self.b.g = out.g.sum(0)

In [132]:
w1.g,b1.g,w2.g,b2.g = [None]*4
model = Model()

In [133]:
%time loss = model(x_train, y_train)

CPU times: user 166 ms, sys: 3.69 ms, total: 170 ms
Wall time: 43.7 ms


In [134]:
%time model.backward()

CPU times: user 513 ms, sys: 83.7 ms, total: 597 ms
Wall time: 154 ms


In [135]:
test_near(w2g, w2.g)
test_near(b2g, b2.g)
test_near(w1g, w1.g)
test_near(b1g, b1.g)
test_near(ig, x_train.g)

### nn.Linear and nn.Module

In [136]:
#export
from torch import nn

In [142]:
class Model(nn.Module):
    def __init__(self, n_in, nh, n_out):
        super().__init__()
        self.layers = [nn.Linear(n_in, nh), nn.ReLU(), nn.Linear(nh, n_out)]
        self.loss = mse
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x.squeeze(), targ)

In [143]:
model = Model(m, nh, 1)

In [144]:
%time loss = model(x_train, y_train)

CPU times: user 142 ms, sys: 5.43 ms, total: 147 ms
Wall time: 41.6 ms


In [145]:
%time loss.backward()

CPU times: user 154 ms, sys: 4.75 ms, total: 159 ms
Wall time: 41.7 ms


## Export

In [146]:
!./notebook2script.py 02_fully_connected_notes.ipynb

Converted 02_fully_connected_notes.ipynb to exp/nb_02.py


# Referenes
* [Matrix Calculus](https://explained.ai/matrix-calculus/index.html)
* [Delving Deep into Rectifiers:
Surpassing Human-Level Performance on ImageNet Classification](https://arxiv.org/pdf/1502.01852.pdf)
* [What is backpropagation really doing? (3Blue1Brown Series)](https://www.youtube.com/watch?v=Ilg3gGewQ5U)